In [1]:
#IMPORTS
import astropy
from astroquery.gaia import Gaia
import numpy
import pandas as pd
Gaia.login(credentials_file='../files/gaia_credentials.txt')

INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


In [2]:
# READ IN GCNS
# the gaia catalogue of nearby stars (GCNS) from edr3
# obtained file at link below
# https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/649/A6#/browse

gcns = astropy.io.ascii.read('../files/table1c.csv')

In [3]:
# CHECK FOR IDS
ids = 'col1' #according to readme https://cdsarc.cds.unistra.fr/ftp/J/A+A/649/A6/ReadMe
print(gcns[ids])

        col1       
-------------------
2334666126716440064
2341871673090078592
 530861741656374272
2745400068346761216
2855176271335676800
4616265759236854016
2875125810310195712
2335003848584711040
2773793203786254208
2740326852975975040
                ...
2415620208636803584
6386702994347886080
2875428141647824000
2421141273132330624
2341766188693540864
2747083180130073344
2766925825958528512
2773791481503524992
6521388186590534272
2011682661920690304
2314850075324449408
Length = 331312 rows


In [4]:
#GET DR3 AND CHECK COLUMN NAMES
dr3_table = Gaia.load_table('gaiadr3.gaia_source')
dr3_mass = Gaia.load_table('gaiadr3.astrophysical_parameters')

for column in dr3_mass.columns:
  print(column.name)

Retrieving table 'gaiadr3.gaia_source'
Retrieving table 'gaiadr3.astrophysical_parameters'
solution_id
source_id
classprob_dsc_combmod_quasar
classprob_dsc_combmod_galaxy
classprob_dsc_combmod_star
classprob_dsc_combmod_whitedwarf
classprob_dsc_combmod_binarystar
classprob_dsc_specmod_quasar
classprob_dsc_specmod_galaxy
classprob_dsc_specmod_star
classprob_dsc_specmod_whitedwarf
classprob_dsc_specmod_binarystar
classprob_dsc_allosmod_quasar
classprob_dsc_allosmod_galaxy
classprob_dsc_allosmod_star
teff_gspphot
teff_gspphot_lower
teff_gspphot_upper
logg_gspphot
logg_gspphot_lower
logg_gspphot_upper
mh_gspphot
mh_gspphot_lower
mh_gspphot_upper
distance_gspphot
distance_gspphot_lower
distance_gspphot_upper
azero_gspphot
azero_gspphot_lower
azero_gspphot_upper
ag_gspphot
ag_gspphot_lower
ag_gspphot_upper
abp_gspphot
abp_gspphot_lower
abp_gspphot_upper
arp_gspphot
arp_gspphot_lower
arp_gspphot_upper
ebpminrp_gspphot
ebpminrp_gspphot_lower
ebpminrp_gspphot_upper
mg_gspphot
mg_gspphot_lower
m

In [5]:
#do it in chunks
CHUNK_SIZE = 2000
max_row = 331311

rows_min = 0
while rows_min < max_row:
    allIds = str(gcns[ids][rows_min])
    n = rows_min + 1

    if(rows_min + CHUNK_SIZE > max_row):
        max_id = max_row
    else:
        max_id = rows_min + CHUNK_SIZE
    
    while(n < max_id):
        allIds = allIds + "," + str(gcns[ids][n])
        n += 1

    query_GCNS = """SELECT
    X.source_id, X.RA, X.RA_ERROR, X.DEC, X.DEC_ERROR, X.PARALLAX, X.PARALLAX_ERROR,
    X.PMRA, X.PMRA_ERROR, X.PMDEC, X.PMDEC_ERROR, X.RADIAL_VELOCITY, X.RADIAL_VELOCITY_ERROR,
    X.phot_g_mean_mag, X.phot_bp_mean_mag, X.phot_rp_mean_mag,
    X.has_mcmc_gspphot,
    X.mh_gspphot, X.mh_gspphot_lower, X.mh_gspphot_upper,
    X.distance_gspphot, X.distance_gspphot_lower, X.distance_gspphot_upper,
    X.teff_gspphot, X.teff_gspphot_lower, X.teff_gspphot_upper,
    X.logg_gspphot, X.logg_gspphot_lower, X.logg_gspphot_upper,
    X.ipd_frac_multi_peak
    FROM gaiadr3.gaia_source X
    WHERE X.source_id IN (""" + allIds +")\n"
    
    query_GCNS_mass = """SELECT
    MASS_FLAME
    FROM gaiadr3.astrophysical_parameters
    WHERE source_id IN (""" + allIds +")"

    job_gcns = Gaia.launch_job(query_GCNS)
    results_chunk = job_gcns.get_results().to_pandas()
    
    job_gcns_mass = Gaia.launch_job(query_GCNS_mass)
    results_chunk_mass = job_gcns_mass.get_results().to_pandas()
    
    results_chunk['mass_flame'] = results_chunk_mass['MASS_FLAME']
    if(rows_min == 0):
        results_gcns = results_chunk
    else:
        results_gcns = pd.concat([results_gcns, results_chunk], ignore_index=True)
    print(results_gcns.shape[0])

    rows_min += CHUNK_SIZE

2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000
130000
132000
134000
136000
138000
140000
142000
144000
146000
148000
150000
152000
154000
156000
158000
160000
162000
164000
166000
168000
170000
172000
174000
176000
178000
180000
182000
184000
186000
188000
190000
192000
194000
196000
198000
200000
202000
204000
206000
208000
210000
212000
214000
216000
218000
220000
222000
224000
226000
228000
230000
232000
234000
236000
238000
240000
242000
244000
246000
248000
250000
252000
254000
256000
258000
260000
262000
264000
266000
268000
270000
272000
274000
276000
278000
280000
282000
284000
286000
288000
290000
292000
294000
296000
298000
300000
302

In [6]:
results_gcns.rename(columns = {'Unnamed: 0':'index'}, inplace=True)
results_gcns.to_csv("../files/gcns_DR3.csv")

# uncomment to save a second csv with only stars with mcmc gspphot from gaia
#results_gcns[results_gcns["has_mcmc_gspphot"]].to_csv("gcns_refined_with_mass.csv")

In [7]:
Gaia.logout()

INFO: Gaia TAP server logout OK [astroquery.gaia.core]
INFO: Gaia data server logout OK [astroquery.gaia.core]
